# Import data

In [2]:
import pandas as pd
import numpy as np
from pandas.api.types import infer_dtype

df = pd.read_csv('./csv_files/properties.csv', low_memory=False)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75876 entries, 0 to 75875
Data columns (total 22 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   source              75876 non-null  int64  
 1   hyperlink           59908 non-null  object 
 2   locality            75874 non-null  object 
 3   postcode            68464 non-null  float64
 4   house_is            75876 non-null  object 
 5   property_subtype    75876 non-null  object 
 6   price               74405 non-null  object 
 7   sale                25810 non-null  object 
 8   rooms_number        74502 non-null  object 
 9   area                72349 non-null  object 
 10  kitchen_has         69763 non-null  object 
 11  furnished           69438 non-null  object 
 12  open_fire           70005 non-null  object 
 13  terrace             61138 non-null  object 
 14  terrace_area        64583 non-null  object 
 15  garden              67421 non-null  object 
 16  gard

# Check if there are columns with mixed data types ==> NO

In [3]:
# print data type of each column to check if there are
# any mixed ones, turns out that there are none
def is_mixed(col):
    return infer_dtype(col)

df.apply(is_mixed)

source                 integer
hyperlink               string
locality                string
postcode              floating
house_is                string
property_subtype        string
price                   string
sale                    string
rooms_number            string
area                    string
kitchen_has            boolean
furnished              boolean
open_fire              boolean
terrace                 string
terrace_area            string
garden                  string
garden_area             string
land_surface            string
land_plot_surface       string
facades_number          string
swimming_pool_has      boolean
building_state          string
dtype: object

# Remove leading and trailing spaces from column names

In [4]:
df.columns = [x.strip(' ') for x in df.columns.values]

# Find which kind of empties there are ==> there are only NaNs

In [5]:
# are there any empty strings? ==> no
print(np.where(df.applymap(lambda x: x == '')))

# are there any NaNs? ==> yes
np.where(pd.isnull(df))

(array([], dtype=int64), array([], dtype=int64))


(array([    0,     1,     2, ..., 75875, 75875, 75875]),
 array([ 7,  7,  7, ..., 19, 20, 21]))

# Display the percent of NaNs per column

In [6]:
# display the percent of NaNs per column
percent_missing = df.isnull().sum() * 100 / len(df)
missing_value_df = pd.DataFrame({'column_name': df.columns,
                                 'percent_missing': percent_missing})
sorted_missing_value_df = missing_value_df.sort_values('percent_missing', ascending = False)
sorted_missing_value_df

,column_name,percent_missing
sale,sale,65.983974
building_state,building_state,41.092045
hyperlink,hyperlink,21.044863
garden_area,garden_area,20.736465
terrace,terrace,19.423797
land_plot_surface,land_plot_surface,16.698297
land_surface,land_surface,15.643945
terrace_area,terrace_area,14.883494
facades_number,facades_number,12.495387
swimming_pool_has,swimming_pool_has,11.986662


# Put 'unknown' in place of NaN for everything else than int64 and float64 columns
## Please run this after converting numerical columns like price and facades from string to integer.  
Even a value of NaN might help predict the price, so to avoid the correlation algorithm skipping it?, and because NaN is not allowed, we replace it.

In [14]:
# replace all NaNs in strings with 'unknown'
df_nanfilled = df.select_dtypes(exclude=['int64','float64']).replace(np.nan, 'unknown')
df.update(df_nanfilled)
df.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75876 entries, 0 to 75875
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   source             75876 non-null  int64  
 1   hyperlink          75876 non-null  object 
 2   locality           75876 non-null  object 
 3   postcode           68464 non-null  float64
 4   house_is           75876 non-null  object 
 5   property_subtype   75876 non-null  object 
 6   price              75876 non-null  object 
 7   sale               75876 non-null  object 
 8   rooms_number       75876 non-null  object 
 9   area               75876 non-null  object 
 10  kitchen_has        75876 non-null  object 
 11  furnished          75876 non-null  object 
 12  open_fire          75876 non-null  object 
 13  terrace            75876 non-null  object 
 14  terrace_area       75876 non-null  object 
 15  garden             75876 non-null  object 
 16  garden_area        758

# Remove leading and trailing spaces in values

In [3]:
# remove leading and trailing spaces and newline characters from values if they are a string
df = df.applymap(lambda x: x.strip() if type(x)==str else x)

# Drop postcode column, because postcode is more completely available in 'locality'
## first we fix 'locality' column to carry just postcode (stripping sporadic address parts)

In [22]:
import re

def clean_locality(locality): 
    # Search for the presence of a 4 digit number 
    if re.search('[1-9]\d{3}', locality):
        # get the number
        return re.findall("[1-9]\d{3}", locality)[0]
    else: 
        # if no postcode is inside insert 'unknown' 
        return 'unknown'
          
# Updated locality column
df['locality'] = df['locality'].apply(clean_locality)

df.drop('postcode', axis = 1, inplace = True)

# Remove duplicates, should execute after editing columns!

In [39]:
# drop 100% duplicate rows,
df.drop_duplicates(ignore_index = True, inplace = True)

,source,hyperlink,locality,postcode,house_is,property_subtype,price,sale,rooms_number,area,...,open_fire,terrace,terrace_area,garden,garden_area,land_surface,land_plot_surface,facades_number,swimming_pool_has,building_state
0,6,8901695,4180,4180.0,True,MIXED_USE_BUILDING,295000,NaN,3,242,...,False,True,36,True,1000,1403,1403,0,False,GOOD
1,6,8747010,8730,8730.0,True,VILLA,675000,NaN,4,349,...,False,False,0,True,977,1526,1526,0,False,AS_NEW
2,6,8775843,4020,4020.0,True,APARTMENT_BLOCK,250000,NaN,5,303,...,False,False,0,False,0,760,760,0,False,TO_RENOVATE
3,6,8910441,1200,1200.0,True,HOUSE,545000,NaN,4,235,...,False,False,0,False,0,63,63,0,False,JUST_RENOVATED
4,6,8758672,1190,1190.0,True,MIXED_USE_BUILDING,500000,NaN,2,220,...,False,False,0,True,60,193,193,0,False,AS_NEW
